# Topic modelling of news headlines for prediction of news category¶

Now we wan't to perform topic modelling with unsupervised learning and train a modell for prediction of news category labels without using the provided labels in the corpus.
We use the LDA classifier on the combined feature 'title_description_text' feature and assume 7 topics. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import joblib

import pyLDAvis.lda_model

## Load data

In [2]:
train = pd.read_csv('../../data/01_train_nosplit_preprocessed.csv')

In [3]:
train.head()

,date,title,description,category,text,source,title_description_text
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...


## Load German stopwords

In [4]:
stop_words = pd.read_csv('german_stopwords.txt', header=None)[0].values.tolist()

In [5]:
print(stop_words)

['ab', 'aber', 'alle', 'allein', 'allem', 'allen', 'aller', 'allerdings', 'allerlei', 'alles', 'allmählich', 'allzu', 'als', 'alsbald', 'also', 'am', 'an', 'and', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'andererseits', 'anderes', 'anderm', 'andern', 'andernfalls', 'anders', 'anstatt', 'auch', 'auf', 'aus', 'ausgenommen', 'ausser', 'ausserdem', 'außer', 'außerdem', 'außerhalb', 'bald', 'bei', 'beide', 'beiden', 'beiderlei', 'beides', 'beim', 'beinahe', 'bereits', 'besonders', 'besser', 'beträchtlich', 'bevor', 'bezüglich', 'bin', 'bis', 'bisher', 'bislang', 'bist', 'bloß', 'bsp.', 'bzw', 'ca', 'ca.', 'content', 'da', 'dabei', 'dadurch', 'dafür', 'dagegen', 'daher', 'dahin', 'damals', 'damit', 'danach', 'daneben', 'dann', 'daran', 'darauf', 'daraus', 'darin', 'darum', 'darunter', 'darüber', 'darüberhinaus', 'das', 'dass', 'dasselbe', 'davon', 'davor', 'dazu', 'daß', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'dem', 'demnach', 'demselben', 'den', 'denen', 'denn',

## Extract features from 'title_description_text' 

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
cv = CountVectorizer(max_df=0.95, min_df=3, stop_words=stop_words)

In [8]:
# create Document-Term-Matrix
dtm = cv.fit_transform(train['title_description_text'])

C:\Users\manuela.rink\anaconda3\envs\nlp\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bsp', 'zb'] not in stop_words.
  warnings.warn(


In [9]:
dtm

<67513x37203 sparse matrix of type '<class 'numpy.int64'>'
	with 1119124 stored elements in Compressed Sparse Row format>

In [10]:
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
lda = LatentDirichletAllocation(n_components=7, random_state=42)
lda.fit(dtm)

LatentDirichletAllocation(n_components=7, random_state=42)

## Analyse extracted features

In [12]:
len(cv.get_feature_names_out())

37203

In [13]:
cv.get_feature_names_out()

array(['00', '000', '007', ..., 'но', 'украинцы', 'это'], dtype=object)

In [14]:
len(lda.components_)

7

In [15]:
lda.components_

array([[1.42857289e-01, 1.70222415e+02, 1.42857227e-01, ...,
        1.42857254e-01, 1.42857265e-01, 1.42857217e-01],
       [1.43072769e-01, 4.84120087e+02, 3.14077412e+00, ...,
        1.42857312e-01, 1.42857328e-01, 1.42857255e-01],
       [1.13438875e+00, 2.21070825e+02, 1.44671435e-01, ...,
        1.42857322e-01, 1.42857339e-01, 1.42857261e-01],
       ...,
       [1.49651046e-01, 2.62251529e+02, 1.14140668e+00, ...,
        1.42857319e-01, 1.42857336e-01, 1.42857259e-01],
       [1.43413041e-01, 1.43093711e-01, 1.43517587e-01, ...,
        1.42857306e-01, 1.42857322e-01, 1.42857251e-01],
       [1.43237738e-01, 2.00219022e+02, 1.42857280e-01, ...,
        4.14285615e+00, 5.14285605e+00, 4.14285648e+00]])

In [16]:
len(lda.components_[0])

37203

### Show most important words of first extracted topic

In [17]:
first_topic = lda.components_[0]

In [18]:
first_topic.argsort() # returns indices of ascending sorted array values 

array([ 9604,  9208, 35080, ..., 26228, 18192, 31472], dtype=int64)

In [19]:
# the last 10 entries are the 10 most important words for topic 1
top_words_indices = first_topic.argsort()[-10:]

In [20]:
for index in top_words_indices:
    print(cv.get_feature_names_out()[index])

kiew
ukrainische
russischen
china
scholz
russische
putin
russland
krieg
ukraine


### Show most important words of first extracted topic

In [21]:
for index, topic in enumerate(lda.components_):
    print(f'Die TOP-15 Wörter für das Thema #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

Die TOP-15 Wörter für das Thema #0
['moskau', 'usa', 'russlands', 'präsident', 'selenskyj', 'kiew', 'ukrainische', 'russischen', 'china', 'scholz', 'russische', 'putin', 'russland', 'krieg', 'ukraine']


Die TOP-15 Wörter für das Thema #1
['geld', 'millionen', 'länder', 'deutsche', 'neue', 'milliarden', 'faeser', 'gibt', 'inflation', 'menschen', 'prozent', 'jahr', 'euro', 'deutschland', 'eu']


Die TOP-15 Wörter für das Thema #2
['gibt', 'deutsche', 'deutschen', 'bahn', 'wegen', 'focus', 'online', 'frankreich', 'news', 'menschen', 'lauterbach', 'pandemie', 'deutschland', 'wm', 'corona']


Die TOP-15 Wörter für das Thema #3
['münchen', 'sieg', 'letzte', 'gestorben', 'berliner', 'fußball', 'ex', 'league', 'generation', 'bundesliga', 'fc', 'jahren', 'iran', 'bayern', 'berlin']


Die TOP-15 Wörter für das Thema #4
['nord', 'klima', 'menschen', 'euro', 'neue', 'energie', 'ampel', 'eu', 'regierung', 'deutschen', 'lindner', 'bundesregierung', 'gas', 'habeck', 'deutschland']


Die TOP-15 Wörte

### Get topics with highest probability for news items in train data

In [22]:
topic_results = lda.transform(dtm)
topic_results.shape

(67513, 7)

In [23]:
topic_results[0].round(5)

array([0.00622, 0.21893, 0.00622, 0.00623, 0.74993, 0.00625, 0.00623])

In [24]:
topic_results[0].argmax()

4

In [25]:
# save topic to train dataset
train['topic'] = topic_results.argmax(axis=1)

In [26]:
train.head()

,date,title,description,category,text,source,title_description_text,topic
0,2022-06-01 00:13:42,Preise: Grüne halten Senkung der Spritsteuer f...,Heute tritt die Steuersenkung auf Kraftstoffe ...,Other,NaN,stern,Preise: Grüne halten Senkung der Spritsteuer f...,4
1,2022-06-01 01:55:03,Biden warnt Putin: USA liefern moderne Raketen...,Die USA rüsten die Ukraine mit fortschrittlich...,Other,NaN,stern,Biden warnt Putin: USA liefern moderne Raketen...,0
2,2022-06-01 02:04:08,Soziale Medien: FDP-Politiker Kuhle: Internet-...,Eine «ZDF Magazin Royale»-Recherche beschäftig...,Other,NaN,stern,Soziale Medien: FDP-Politiker Kuhle: Internet-...,5
3,2022-06-01 02:26:58,Liveblog: ++ Zwei von drei ukrainischen Kinder...,Rund zwei von drei Mädchen und Jungen in der U...,Missing,NaN,Tagesschau,Liveblog: ++ Zwei von drei ukrainischen Kinder...,0
4,2022-06-01 02:31:43,Finanzen: Dänemark stimmt über EU-Verteidigung...,Vorbehalt verteidigen oder Verteidigung ohne V...,Other,NaN,stern,Finanzen: Dänemark stimmt über EU-Verteidigung...,0


## Visualize model

In [27]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, dtm, cv, mds='tsne')
panel

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      28.314230 -11.381309       1        1  20.829600
5      -7.893435  -6.939400       2        1  14.720612
1      -0.757814 -23.699026       3        1  13.856477
4      21.144053   5.364448       4        1  13.411521
6      10.181520  -9.144038       5        1  12.866064
2      17.304569 -25.899475       6        1  12.668150
3       3.044635   7.625439       7        1  11.647576, topic_info=           Term          Freq         Total Category  logprob  loglift
31472   ukraine  11641.000000  11641.000000  Default  30.0000  30.0000
18192     krieg   5292.000000   5292.000000  Default  29.0000  29.0000
26228  russland   5136.000000   5136.000000  Default  28.0000  28.0000
6556     corona   2890.000000   2890.000000  Default  27.0000  27.0000
9760         eu   4126.000000   4126.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
24024   polizei    370.968280   1313.341350   Topic7  -5.9758   0.8859
34719     wegen    371.638788   2713.687909   Topic7  -5.9740   0.1619
12189      geht    365.056890   2875.555223   Topic7  -5.9919   0.0861
35689    worden    338.862058   1769.644895   Topic7  -6.0664   0.4971
36592      zwei    317.591901   2497.251777   Topic7  -6.1312   0.0879

[491 rows x 6 columns], token_table=       Topic      Freq              Term
term                                    
1          1  0.125349               000
1          3  0.357813               000
1          4  0.194480               000
1          5  0.146620               000
1          6  0.161054               000
...      ...       ...               ...
36838      3  0.026844                öl
36838      4  0.883159                öl
36891      2  0.008223           özdemir
36891      3  0.986797           özdemir
37100      4  0.993628  überschwemmungen

[1122 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 2, 5, 7, 3, 4])

## Save train set with topics and dump countvectorizer and model

In [28]:
#Save result to csv
train.to_csv('train_lda_combined.csv')

In [29]:
# Dump countvectorizer
joblib.dump(cv, 'cv_combined.jl')

['cv_combined.jl']

In [30]:
# Dump LDA model
joblib.dump(lda, 'lda_model_7_topics_combined.jl')

['lda_model_7_topics_combined.jl']